### 1. Configuração Inicial

**Objetivo:** Importar as bibliotecas e definir os caminhos e nomes de tabelas.

In [0]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType
from pyspark.sql.functions import col, input_file_name, current_timestamp

# --- CONFIGURAÇÃO DOS CAMINHOS E NOMES ---
data_source = "sinasc"
root_path = "/lakehouse/health_insights_brasil"
# Apontamos para o diretório da AMOSTRA que criamos no notebook anterior
landing_zone_path = f"{root_path}/landing_zone/{data_source}" 
bronze_path = f"{root_path}/bronze/{data_source}"
table_name = "bronze_sinasc"

print("✅ Configuração concluída.")
print(f"   - Lendo de: {landing_zone_path}")
print(f"   - Escrevendo em: {bronze_path}")

✅ Configuração concluída.
   - Lendo de: /lakehouse/health_insights_brasil/landing_zone/sinasc
   - Escrevendo em: /lakehouse/health_insights_brasil/bronze/sinasc


### 2. Definição Explícita do Schema

**Objetivo:** Criar o "contrato" dos nossos dados. Definimos cada coluna e seu tipo. Para a camada Bronze, é uma boa prática ler a maioria dos campos como `StringType` para evitar erros de parse. A conversão para números ou datas será feita na camada Silver.

In [0]:
# Schema explícito para os dados do SINASC 
bronze_schema = StructType([
    StructField("ORIGEM", StringType(), True),
    StructField("CODESTAB", StringType(), True),
    StructField("CODMUNNASC", StringType(), True),
    StructField("NUMERODN", StringType(), True), 
    StructField("LOCNASC", StringType(), True),
    StructField("IDADEMAE", StringType(), True),
    StructField("ESTCIVMAE", StringType(), True),
    StructField("ESCMAE", StringType(), True),
    StructField("CODOCUPMAE", StringType(), True),
    StructField("QTDFILVIVO", StringType(), True),
    StructField("QTDFILMORT", StringType(), True),
    StructField("CODMUNRES", StringType(), True),
    StructField("GESTACAO", StringType(), True),
    StructField("GRAVIDEZ", StringType(), True),
    StructField("PARTO", StringType(), True),
    StructField("CONSULTAS", StringType(), True),
    StructField("DTNASC", StringType(), True),
    StructField("HORANASC", StringType(), True),
    StructField("SEXO", StringType(), True),
    StructField("APGAR1", StringType(), True),
    StructField("APGAR5", StringType(), True),
    StructField("RACACOR", StringType(), True),
    StructField("PESO", StringType(), True),
    StructField("IDANOMAL", StringType(), True),
    StructField("DTCADASTRO", StringType(), True),
    StructField("CODANOMAL", StringType(), True),
    StructField("NUMEROLOTE", StringType(), True),
    StructField("VERSAOSIST", StringType(), True),
    StructField("DTRECEBIM", StringType(), True),
    StructField("DIFDATA", StringType(), True),
    StructField("DTRECORIGA", StringType(), True),
    StructField("NATURALMAE", StringType(), True),
    StructField("CODMUNNATU", StringType(), True),
    StructField("CODUFNATU", StringType(), True),
    StructField("ESCMAE2010", StringType(), True),
    StructField("SERIESCMAE", StringType(), True),
    StructField("DTNASCMAE", StringType(), True),
    StructField("RACACORMAE", StringType(), True),
    StructField("QTDGESTANT", StringType(), True),
    StructField("QTDPARTNOR", StringType(), True),
    StructField("QTDPARTCES", StringType(), True),
    StructField("IDADEPAI", StringType(), True),
    StructField("DTULTMENST", StringType(), True),
    StructField("SEMAGESTAC", StringType(), True),
    StructField("TPMETESTIM", StringType(), True),
    StructField("CONSPRENAT", StringType(), True),
    StructField("MESPRENAT", StringType(), True),
    StructField("TPAPRESENT", StringType(), True),
    StructField("STTRABPART", StringType(), True),
    StructField("STCESPARTO", StringType(), True),
    StructField("TPNASCASSI", StringType(), True),
    StructField("TPFUNCRESP", StringType(), True),
    StructField("TPDOCRESP", StringType(), True),
    StructField("DTDECLARAC", StringType(), True),
    StructField("ESCMAEAGR1", StringType(), True),
    StructField("STDNEPIDEM", StringType(), True),
    StructField("STDNNOVA", StringType(), True),
    StructField("CODPAISRES", StringType(), True),
    StructField("TPROBSON", StringType(), True),
    StructField("PARIDADE", StringType(), True),
    StructField("KOTELCHUCK", StringType(), True),
    StructField("CONTADOR", StringType(), True),
    StructField("munResUf", StringType(), True),
    StructField("munResNome", StringType(), True),
    StructField("munResLat", StringType(), True),
    StructField("munResLon", StringType(), True),
    StructField("munResAlt", StringType(), True),
    StructField("munResArea", StringType(), True)
])

print("✅ Schema da camada Bronze corrigido com a coluna NUMERODN.")

✅ Schema da camada Bronze corrigido com a coluna NUMERODN.


### 3. Leitura do CSV e Escrita da Tabela Bronze

**Objetivo:** Ler o CSV da Landing Zone forçando o uso do nosso schema, adicionar colunas de metadados e salvar a tabela final no formato Delta.

In [0]:
# --- LIMPEZA PARA UMA CARGA COMPLETA ---
print("Limpando diretórios e tabela antiga para garantir uma carga completa...")
dbutils.fs.rm(bronze_path, recurse=True)
spark.sql(f"DROP TABLE IF EXISTS default.{table_name}")

# --- PROCESSAMENTO EM LOTE ---
print("Iniciando o processamento em lote...")

# Leitura do CSV aplicando nosso schema explícito
df_raw = (spark.read
          .format("csv")
          .option("header", "true")
          .option("sep", ";")
          .schema(bronze_schema) # <-- APLICAÇÃO DO SCHEMA
          .load(landing_zone_path)
         )

# Adiciona as colunas de metadados para rastreabilidade
df_bronze = (df_raw
             .withColumn("_ingestion_timestamp", current_timestamp())
             .withColumn("_source_file", input_file_name())
            )

# Escrita dos dados no formato Delta
print(f"Salvando {df_bronze.count()} registros no formato Delta em: {bronze_path}")
(df_bronze.write
 .format("delta")
 .mode("overwrite")
 .save(bronze_path)
)

# Criação da tabela externa no Metastore
print(f"Registrando a tabela externa '{table_name}'...")
spark.sql(f"""
    CREATE TABLE default.{table_name}
    USING DELTA
    LOCATION '{bronze_path}'
""")

print(f"\n✅ Tabela '{table_name}' criada com sucesso na camada Bronze.")

Limpando diretórios e tabela antiga para garantir uma carga completa...
Iniciando o processamento em lote...
Salvando 10000 registros no formato Delta em: /lakehouse/health_insights_brasil/bronze/sinasc
Registrando a tabela externa 'bronze_sinasc'...

✅ Tabela 'bronze_sinasc' criada com sucesso na camada Bronze.


### 4. Verificação Final

**Objetivo:** Confirmar visualmente que a tabela foi criada e que os dados estão estruturados corretamente com as colunas que definimos.

In [0]:
%sql
-- A verificação final que agora deve mostrar os dados corretamente preenchidos
SELECT * FROM default.bronze_sinasc LIMIT 10;

ORIGEM,CODESTAB,CODMUNNASC,NUMERODN,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,CODOCUPMAE,QTDFILVIVO,QTDFILMORT,CODMUNRES,GESTACAO,GRAVIDEZ,PARTO,CONSULTAS,DTNASC,HORANASC,SEXO,APGAR1,APGAR5,RACACOR,PESO,IDANOMAL,DTCADASTRO,CODANOMAL,NUMEROLOTE,VERSAOSIST,DTRECEBIM,DIFDATA,DTRECORIGA,NATURALMAE,CODMUNNATU,CODUFNATU,ESCMAE2010,SERIESCMAE,DTNASCMAE,RACACORMAE,QTDGESTANT,QTDPARTNOR,QTDPARTCES,IDADEPAI,DTULTMENST,SEMAGESTAC,TPMETESTIM,CONSPRENAT,MESPRENAT,TPAPRESENT,STTRABPART,STCESPARTO,TPNASCASSI,TPFUNCRESP,TPDOCRESP,DTDECLARAC,ESCMAEAGR1,STDNEPIDEM,STDNNOVA,CODPAISRES,TPROBSON,PARIDADE,KOTELCHUCK,CONTADOR,munResUf,munResNome,munResLat,munResLon,munResAlt,munResArea,_ingestion_timestamp,_source_file
1,1,2516500,110001,1,24,1,3,999992,01,00,110001,5,1,2,2,14022024,0845,1,08,09,4,3120,2,05032024,null,20240003,3.2.50,05032024,020,20,05032024,811,110028,11,2,5,11012000,4,01,00,01,null,17052023,38,8,02,03,1,2,2,1,2,3,14022024,03,0,1,1,05,1,2,null,null,null,null,null,null,2025-08-14T21:31:20.819+0000,dbfs:/lakehouse/health_insights_brasil/landing_zone/sinasc/SINASC_2024_SAMPLE_10000.csv
2,1,2516500,110001,1,29,2,5,999992,00,00,110001,5,1,2,4,17042024,0850,1,08,09,4,3564,2,09052024,null,20240005,3.2.50,09052024,022,22,09052024,811,110001,11,5,null,22111994,4,00,00,00,41,17072023,39,8,08,05,1,2,1,1,2,3,17042024,08,0,1,1,02,0,2,null,null,null,null,null,null,2025-08-14T21:31:20.819+0000,dbfs:/lakehouse/health_insights_brasil/landing_zone/sinasc/SINASC_2024_SAMPLE_10000.csv
3,1,2516500,110001,1,30,5,4,622020,02,00,110001,5,1,2,4,29052024,1006,1,08,09,4,2816,2,03062024,null,20240006,3.2.50,03062024,005,5,03062024,811,110001,11,3,2,12081993,4,02,00,02,35,27082023,39,8,08,02,1,2,1,1,2,3,29052024,05,0,1,1,05,1,5,null,null,null,null,null,null,2025-08-14T21:31:20.819+0000,dbfs:/lakehouse/health_insights_brasil/landing_zone/sinasc/SINASC_2024_SAMPLE_10000.csv
4,1,2516500,110001,1,14,5,4,999991,00,00,110001,5,1,2,4,27052024,0900,1,08,09,1,3126,2,03062024,null,20240006,3.2.50,03062024,007,7,03062024,811,110001,11,2,8,12102009,1,00,00,00,17,28082023,38,8,10,03,1,2,2,1,2,3,27052024,04,0,1,1,01,0,5,null,null,null,null,null,null,2025-08-14T21:31:20.819+0000,dbfs:/lakehouse/health_insights_brasil/landing_zone/sinasc/SINASC_2024_SAMPLE_10000.csv
5,1,2516500,110001,1,24,2,4,999992,01,00,110001,5,1,2,4,13052024,0835,1,08,09,4,3622,2,03062024,null,20240006,3.2.50,03062024,021,21,03062024,850,500270,50,3,1,23091999,4,01,00,01,29,08082023,39,8,11,02,1,2,2,1,2,3,13052024,05,0,1,1,05,1,5,null,null,null,null,null,null,2025-08-14T21:31:20.819+0000,dbfs:/lakehouse/health_insights_brasil/landing_zone/sinasc/SINASC_2024_SAMPLE_10000.csv
6,1,2516500,110001,1,29,5,5,999992,00,00,110001,5,1,2,4,08052024,0915,2,08,09,2,2935,2,03062024,null,20240006,3.2.50,03062024,026,26,03062024,811,110014,11,4,null,05121994,2,00,00,00,30,01082023,40,8,07,04,1,2,2,1,2,3,08052024,07,0,1,1,01,0,2,null,null,null,null,null,null,2025-08-14T21:31:20.819+0000,dbfs:/lakehouse/health_insights_brasil/landing_zone/sinasc/SINASC_2024_SAMPLE_10000.csv
7,1,2679477,110001,1,18,9,3,999992,01,00,110001,5,1,1,3,01052024,0110,1,08,09,4,3365,2,03062024,null,20240007,3.2.50,12062024,042,33,03062024,811,110001,11,1,4,28052005,4,01,01,00,null,05082023,38,8,05,01,1,2,3,1,5,5,01052024,02,0,1,1,03,1,3,null,null,null,null,null,null,2025-08-14T21:31:20.819+0000,dbfs:/lakehouse/health_insights_brasil/landing_zone/sinasc/SINASC_2024_SAMPLE_10000.csv
8,1,2516500,110001,1,29,1,4,999992,02,01,110001,5,1,2,4,05062024,1005,1,08,09,4,3298,2,02072024,null,20240008,3.2.50,20082024,076,76,20082024,811,110001,11,3,3,10111994,4,03,00,02,null,05092023,39,8,07,02,1,2,1,1,2,3,05062024,06,0,1,1,05,1,5,null,null,null,null,null,null,2025-08-14T21:31:20.819+0000,dbfs:/lakehouse/health_insights_brasil/landing_zone/sinasc/SINASC_2024_SAMPLE_10000.csv
9,1,2494299,110002,1,20,5,4,999992,00,00,110002,5,1,1,4,01012024,0516,1,08,09,1,3240,2,03012024,null,20240001,3.2.50,09012024,008,8,09012024,811,110002,11,3,3,24062003,1,00,00,00,null,05042023,38,